In [ ]:
import tkinter as tk
from tkinter import ttk
import serial
import time
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import threading

class GlottalStopApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Glottal Stop Calibration")

        # Variables
        self.recording = False
        self.calibrating = False
        self.data = []
        self.rate = tk.DoubleVar(value=1.0)
        self.glottal_stop_range = tk.DoubleVar(value=0.0)
        self.glottal_stop = None
        self.serial_port = serial.Serial('COM3', 9600)  # Adjust COM port as necessary

        # Widgets
        self.create_widgets()

    def create_widgets(self):
        # Text boxes and labels
        tk.Label(self.root, text="Rate:").grid(row=0, column=0)
        tk.Entry(self.root, textvariable=self.rate).grid(row=0, column=1)

        tk.Label(self.root, text="Glottal Stop Range:").grid(row=1, column=0)
        tk.Entry(self.root, textvariable=self.glottal_stop_range).grid(row=1, column=1)

        # Buttons
        ttk.Button(self.root, text="Calibrate Start", command=self.calibrate_start).grid(row=2, column=0)
        ttk.Button(self.root, text="Calibrate Stop", command=self.calibrate_stop).grid(row=2, column=1)
        ttk.Button(self.root, text="Generic Start", command=self.generic_start).grid(row=3, column=0)
        ttk.Button(self.root, text="Generic Stop", command=self.generic_stop).grid(row=3, column=1)
        ttk.Button(self.root, text="Plot Calibrated Glottal Stop", command=self.plot_calibrated_glottal_stop).grid(row=4, column=0, columnspan=2)

        # Plot area
        self.figure, self.ax = plt.subplots()
        self.canvas = FigureCanvasTkAgg(self.figure, self.root)
        self.canvas.get_tk_widget().grid(row=5, column=0, columnspan=2)

    def read_serial_data(self):
        while self.recording or self.calibrating:
            if self.serial_port.in_waiting > 0:
                data = self.serial_port.readline().decode('utf-8').strip()
                if data.isnumeric():
                    self.data.append(float(data))
                time.sleep(1 / self.rate.get())

    def calibrate_start(self):
        self.calibrating = True
        self.data.clear()
        threading.Thread(target=self.read_serial_data).start()

    def calibrate_stop(self):
        self.calibrating = False
        time.sleep(1)  # Give time for the thread to finish
        self.plot_data()

    def generic_start(self):
        self.recording = True
        self.data.clear()
        threading.Thread(target=self.read_serial_data).start()

    def generic_stop(self):
        self.recording = False
        time.sleep(1)  # Give time for the thread to finish
        self.plot_data()

    def plot_data(self):
        self.ax.clear()
        self.ax.plot(self.data)
        self.ax.set_xlabel('Time')
        self.ax.set_ylabel('Voltage')
        self.canvas.draw()

    def plot_calibrated_glottal_stop(self):
        if self.glottal_stop is None:
            glottal_stop_index = int(self.glottal_stop_range.get() * self.rate.get())
            if 0 <= glottal_stop_index < len(self.data):
                self.glottal_stop = self.data[glottal_stop_index]
                self.ax.axvline(x=glottal_stop_index, color='r', linestyle='--')
                self.canvas.draw()
        else:
            for i, value in enumerate(self.data):
                if abs(value - self.glottal_stop) < 0.1:  # Adjust tolerance as needed
                    self.ax.axvline(x=i, color='g', linestyle='--')
            self.canvas.draw()

if __name__ == "__main__":
    root = tk.Tk()
    app = GlottalStopApp(root)
    root.mainloop()
